1. Update config/config.yaml
2. Update secrets.yaml [Optional]
3. Update params.yaml
4. Update ENTITY: src/KDClassification/entity/config-entity.py
5. Update ConfigurationManager: src/KDClassification/config/configuration.py
6. Update components: src/KDClassification/components/data_ingestion.py
7. Update pipeline 
8. Update main.py
<!-- 9. Update dvc.yaml
10. app.py -->

In [1]:
import os
os.chdir("../")

### 4. Update `entity`: `entity/config-entity.py`

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### 5. Update `ConfigurationManager`: `src/KDClassification/config/configuration.py`

In [6]:
from KDClassification.constants import *
from KDClassification.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### 6. Update `components`: `src/KDClassification/components/data_ingestion.py`

In [7]:
import os
import zipfile
import gdown
from KDClassification import logger
from KDClassification.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



### 7. Update `pipeline`: `src/KDClassification/pipeline/stage_01_data_ingestion.py`

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-23 14:06:30,992: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-23 14:06:31,024: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-23 14:06:31,028: INFO: common: Created directory at: artifacts]
[2024-07-23 14:06:31,031: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-07-23 14:06:31,034: INFO: 2532008400: Downloading data from https://drive.google.com/file/d/1jy4gYrs9o2R2Uvgvgqj-r7uteLAdB_Ci/view?usp=sharing into file artifacts/data_ingestion/KDC-data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1jy4gYrs9o2R2Uvgvgqj-r7uteLAdB_Ci
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1jy4gYrs9o2R2Uvgvgqj-r7uteLAdB_Ci&confirm=t&uuid=b2f9d488-342d-49a7-b2f9-205745aee6a1
To: c:\Users\PARIDHI\Desktop\ALL-ML-PROJECTS\Kidney-Disease-Classification\artifacts\data_ingestion\KDC-data.zip
100%|██████████| 941M/941M [01:34<00:00, 9.98MB/s] 


[2024-07-23 14:08:11,295: INFO: 2532008400: Downloaded data from https://drive.google.com/file/d/1jy4gYrs9o2R2Uvgvgqj-r7uteLAdB_Ci/view?usp=sharing into file artifacts/data_ingestion/KDC-data.zip]
